In [1]:
# Importing necessary libraries
import pandas as pd

### Reading and merging data

In [49]:
proy_pop = pd.read_csv('./../datos/info/proy_pop200125.csv', index_col=0)
radio_ref = pd.read_csv('./../datos/info/radio_ref.csv', index_col=0) 
p02 = pd.read_csv('./../datos/info/PERSONA-P02.csv', index_col=0, encoding = 'latin-1')
radios = radio_ref.merge(p02.reset_index())

## Agrupar por DPTO y Eleccion

In [50]:
dptos_p2010 = radios.groupby(['DPTO', 'NOMDPTO'])[p02.columns].sum().reset_index()

### Definir Variables Agrupadoras

In [51]:
# ## Los circuitos vienen identificados por las siguientes columnas
# gr_circuito = ['distrito_id', 'circuito_id']
# ## Las elecciones vienen identificadas por lo siguiente:
# gr_eleccion = ['año', 'eleccion_tipo', 'recuento_tipo', 'padron_tipo']
# gr_secprov = ['seccionprovincial_id']
# gr_seccion = ['distrito_id', 'seccion_id']


## Electores por circuito y por DPTO

In [52]:
# gr_seccion

In [107]:
# electores_circ = pd.read_csv('./../datos/out/n_electores_circ.csv')
# electores_circ.seccion_id.nunique()

electores_dpto = pd.read_csv('./../datos/out/n_electores_dpto.csv')
electores_dpto = electores_dpto.dropna()
electores_dpto.columns = ['eleccion_id', 'distrito_id', 'seccion_id', 'mesa_electores_sum', 'mesa_electores_size']
electores_dpto = electores_dpto.astype(float).astype(int)

secciones_departamentos = pd.read_csv('./../datos/info/secciones_dptos_ref.csv')

p2010 = secciones_departamentos.merge(dptos_p2010[['DPTO', 'TOTAL']], how = 'left')[['distrito_id', 'seccion_id', 'TOTAL']]
p2010
# # secciones_departamentos = pd.read_csv('./../datos/info/secciones_departamentos-TTGL.csv')
# secciones = pd.read_csv('./../datos/BD/seccion_table.csv')
# secciones.sort_values(['distrito_id', 'seccion_id']).to_csv('./../datos/info/secciones_dptos_ref_.csv', index = False)

,distrito_id,seccion_id,TOTAL
0,1,1,205886.0
1,1,2,157932.0
2,1,3,187537.0
3,1,4,218245.0
4,1,5,179005.0
...,...,...,...
527,23,17,14933.0
528,24,1,56956.0
529,24,2,70042.0
530,24,3,NaN


In [114]:
electores_p2010 = electores_dpto.merge(p2010)
electores_p2010['ratio'] = electores_p2010['TOTAL']/electores_p2010['mesa_electores_sum']

In [117]:
electores_p2010.groupby('eleccion_id')['ratio'].mean()

eleccion_id
0     1.412283
1     2.952990
2     1.288456
3     1.291557
4     1.228286
5     1.284108
6     1.244751
7     1.166222
8     1.166222
9     1.164427
10    1.164427
11    1.218363
12    1.198087
13    9.328880
14    8.833853
15    1.161315
16    1.161315
Name: ratio, dtype: float64

## Outliers

In [ ]:
outliers = abs(electores.div(p2010, axis = 0).mean(1) - 1) > .25
dptos = radio_ref[['PROV_REF_ID','IDDPTO', 'DPTO', 'NOMPROV', 'NOMDPTO']].drop_duplicates().set_index('DPTO')
df = electores.div(p2010, axis = 0)
dptos_nam_elecs = dptos.merge(df, left_index = True, right_index = True)
large = electores.mean(1) > electores.mean(1).quantile(.7)
sorter = ('mesa_electores_sum', 2011, 'GENERAL', 'PROVISORIO', 'NORMAL')
dptos_nam_elecs.loc[(outliers == True) & (large == True)].round(3).sort_values(sorter)


ValueError: cannot join with no overlapping index names

In [ ]:
## Hay Problema con misiones y san juan
# dptos_nam_elecs = dptos.merge(electores.loc[outliers == True], left_index = True, right_index = True)

# df = electores
df = electores.div(p2010, axis = 0)

dptos_nam_elecs = dptos.merge(df, left_index = True, right_index = True)

# san juan con problema de id en algunas elecciones 2019. Misiones con problema de id en ..
# Jujuy	Rinconada	1441.0	NaN

# electores.div(p2010, axis = 0).loc[outliers == True]

# dptos_nam_elecs.loc[outliers == True].round(3).to_csv('./dptos_nam_elecs_outliers.csv')


large = electores.mean(1) > electores.mean(1).quantile(.7)

# Cuantos lugares con mas de 70% de la media
# large.sum()

# Cuanto representan
# electores.loc[large == True].sum()/electores.sum() 

sorter = ('mesa_electores_sum', 2011, 'GENERAL', 'PROVISORIO', 'NORMAL')
dptos_nam_elecs.loc[(outliers == True) & (large == True)].round(3).sort_values(sorter)


/tmp/ipykernel_6929/2705720179.py:7: FutureWarning: merging between different levels is deprecated and will be removed in a future version. (1 levels on the left, 5 on the right)
  dptos_nam_elecs = dptos.merge(df, left_index = True, right_index = True)


,PROV_REF_ID,IDDPTO,NOMPROV,NOMDPTO,"(mesa_electores_sum, 2011, GENERAL, PROVISORIO, NORMAL)","(mesa_electores_sum, 2011, PASO, PROVISORIO, NORMAL)","(mesa_electores_sum, 2013, GENERAL, PROVISORIO, NORMAL)","(mesa_electores_sum, 2013, PASO, PROVISORIO, NORMAL)","(mesa_electores_sum, 2015, BALLOTAGE, PROVISORIO, NORMAL)","(mesa_electores_sum, 2015, GENERAL, PROVISORIO, NORMAL)",...,"(mesa_electores_sum, 2017, GENERAL, PROVISORIO, COMANDO)","(mesa_electores_sum, 2017, GENERAL, PROVISORIO, NORMAL)","(mesa_electores_sum, 2017, PASO, PROVISORIO, COMANDO)","(mesa_electores_sum, 2017, PASO, PROVISORIO, NORMAL)","(mesa_electores_sum, 2019, GENERAL, PROVISORIO, COMANDO)","(mesa_electores_sum, 2019, GENERAL, PROVISORIO, NORMAL)","(mesa_electores_sum, 2019, PASO, PROVISORIO, COMANDO)","(mesa_electores_sum, 2019, PASO, PROVISORIO, NORMAL)","(mesa_electores_sum, 2021, PASO, PROVISORIO, COMANDO)","(mesa_electores_sum, 2021, PASO, PROVISORIO, NORMAL)"
DPTO,,,,,,,,,,,,,,,,,,,,,
82084,21,84,Santa Fe,Rosario,0.175,0.134,0.180,0.180,0.855,0.849,...,0.863,0.863,0.865,0.865,0.849,0.860,0.867,0.867,0.852,0.852
54091,14,91,Misiones,Oberá,0.366,0.273,0.407,0.407,0.448,0.446,...,0.489,0.489,0.491,0.491,0.506,0.506,0.509,0.509,0.525,0.525
6427,2,427,Buenos Aires,La Matanza,0.493,0.360,0.537,0.536,0.528,0.559,...,0.612,0.612,0.612,0.612,0.551,0.605,0.566,0.622,0.626,0.626
70028,18,28,San Juan,Capital,0.514,0.226,0.579,0.578,0.607,0.597,...,0.639,0.639,0.642,0.642,NaN,0.640,0.643,0.643,0.663,0.663
38014,10,14,Jujuy,El Carmen,0.560,0.195,0.636,0.635,0.679,0.626,...,0.754,0.754,0.754,0.754,NaN,0.781,0.761,0.761,0.823,0.823
6638,2,638,Buenos Aires,Pilar,0.569,0.386,0.645,0.643,0.693,0.724,...,0.813,0.813,0.813,0.813,0.763,0.804,0.598,0.602,0.910,0.910
90063,23,63,Tucumán,Lules,0.574,0.174,0.629,0.630,0.711,0.701,...,0.747,0.747,0.748,0.748,0.762,0.762,0.772,0.772,0.798,0.798
6270,2,270,Buenos Aires,Ezeiza,0.576,0.441,0.655,0.653,0.697,0.717,...,0.806,0.806,0.806,0.806,0.742,0.801,0.783,0.841,0.869,0.869
6274,2,274,Buenos Aires,Florencio Varela,0.597,0.361,0.656,0.654,0.666,0.667,...,0.758,0.758,0.758,0.758,0.702,0.740,0.728,0.770,0.776,0.776
